In [25]:
!pip install -qq google-cloud-texttospeech
!pip freeze | grep google-cloud-texttospeech | awk -F '=' '{print $NF}'

2.2.0


In [26]:
import numpy as np
import IPython.display as ipd
from google.cloud import texttospeech
import time

In [27]:
%%capture --no-stderr
# https://console.cloud.google.com/apis/credentials/serviceaccountkey
%set_env GOOGLE_APPLICATION_CREDENTIALS=My First Project-c680cc2bdc4b.json

In [28]:
LANGUAGE='ru'

In [29]:
client = texttospeech.TextToSpeechClient()
cloud_voices = list(voice.name for voice in client.list_voices(language_code=LANGUAGE).voices)
print(len(cloud_voices))

10


In [30]:
def speech_synth(text, cloud_voice, speaking_rate):

  # Select the type of audio file you want returned
  audio_config = texttospeech.AudioConfig(
      audio_encoding=texttospeech.AudioEncoding.LINEAR16,
      sample_rate_hertz=16000,
      speaking_rate=speaking_rate)

  # Set the text input to be synthesized
  synthesis_input = texttospeech.SynthesisInput(text=text)
  voice = texttospeech.VoiceSelectionParams(
      language_code=LANGUAGE,
      name=cloud_voice)

  # Perform the text-to-speech request on the text input with the selected
  # voice parameters and audio file type
  response = client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

  time.sleep(1) # throttling

  return response

In [31]:
dataset = ('ноль', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь', 'девять')

In [32]:
%%time
dataset = ([d, d + '?'] for d in dataset)
dataset = ([(w, v) for v in np.arange(0.8, 1.2, 0.025) for w in d] for d in dataset)
dataset = ([speech_synth(w, v, r) for v in cloud_voices for w, r in d] for d in dataset)
dataset = ([s.audio_content for s in d] for d in dataset)
dataset = ([a[44:] for a in d] for d in dataset)
dataset = ([np.frombuffer(a, dtype=np.int16) for a in d] for d in dataset)
dataset = ([np.trim_zeros(a) for a in d] for d in dataset)
dataset = ([a for a in d if len(a) <= 16000 and len(a)] for d in dataset)
dataset = ([np.concatenate(([16000 - len(a)], a)) for a in d] for d in dataset)
dataset = ([np.concatenate((a, [0] * a[0])) for a in d] for d in dataset)
dataset = list(dataset)

CPU times: user 28.5 s, sys: 1.72 s, total: 30.2 s
Wall time: 1h 15min 45s


In [33]:
for i in dataset:
  ipd.display(ipd.Audio(i[0][1:], rate=16000))

In [34]:
for i in dataset:
  ipd.display(ipd.Audio(i[1][1:], rate=16000))

In [35]:
slen = sorted(len(a) for a in dataset)
assert slen[-1] / slen[0] < 1.2, slen
slen

[320, 320, 320, 320, 320, 320, 320, 320, 320, 320]

In [36]:
np.save('synth.npy', dataset)

In [37]:
!gzip -9 'synth.npy'
!md5sum 'synth.npy.gz'

0499495dda29380ec881cbdcca284b2d  synth.npy.gz
